In [4]:
cd '/home/schlecker/repos/planeteScripts'

/home/schlecker/repos/planeteScripts


In [5]:
%load_ext autoreload
%autoreload 2
from make_list import *
from planeteOutput import *
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [182]:
import seaborn as sns

##### Settings for papers/theses:
# Set plot style
sns.set(context='notebook', style='ticks', font_scale=1.3, palette='pastel',\
    rc={
'text.usetex':True,
'text.latex.unicode':True,
'font.family' : 'sans-serif',
#'font.serif':'Computer Modern',
'font.style'         : 'normal',
'font.variant'        : 'normal',
'font.weight'         : 'normal',
'font.stretch'        : 'normal',
'savefig.dpi'         : 400,
'lines.linewidth'     : 1.0,  
'figure.subplot.left'    : 0.14,    # the left side of the subplots of the figure
'figure.subplot.right'   : 0.96,   # the right side of the subplots of the figure
'figure.subplot.bottom'  : 0.13,   # the bottom of the subplots of the figure
'figure.subplot.top'     : 0.96,    # the top of the subplots of the figure
'figure.subplot.hspace'  : 0.0,    # height reserved for space between subplots
'axes.xmargin' : 0.02,             # default margin for autoscale
'axes.ymargin' : 0.02,
'legend.handletextpad' : 0.5,
'legend.handlelength' : 0.75,
'xtick.minor.size'     : 2.,
'ytick.minor.size'     : 2.
})


In [6]:
# load an example population
popfile = '/home/schlecker/phd/planete/outputs/bernNov17/popu/popu.hdf5'
j35file = '/home/schlecker/phd/planete/outputs/J35/pop04f_J35.hd5'
pop = read_popHdf5(j35file, hierarchical=False, nSample=5)

SIM1515
SIM6010
SIM8511
SIM9792
SIM9845


In [31]:
T1simFile = '/home/schlecker/phd/planete/outputs/singleruns/01Trappist1/'
T1sim = read_simFromFolder(T1simFile)

Reading files: /home/schlecker/phd/planete/outputs/singleruns/01Trappist1//tracks*.outputdat
Reading planet001...
Reading planet002...
Reading planet003...
Reading planet004...
Reading planet005...
Reading planet006...
Reading planet007...
Reading planet008...
Reading planet009...
Reading planet010...
Reading planet011...
Reading planet012...
Reading planet013...
Reading planet014...
Reading planet015...
Reading planet016...
Reading planet017...
Reading planet018...
Reading planet019...
Reading planet020...
Reading planet021...
Reading planet022...
Reading planet023...
Reading planet024...
Reading planet025...
Reading planet026...
Reading planet027...
Reading planet028...
Reading planet029...
Reading planet030...
Reading planet031...
Reading planet032...
Reading planet033...
Reading planet034...
Reading planet035...
Reading planet036...
Reading planet037...
Reading planet038...
Reading planet039...
Reading planet040...
Reading planet041...
Reading planet042...
Reading planet043...
Read

In [82]:
T1sim['planet013'].status.tail(1) == -1

39920    False
Name: status, dtype: bool

In [87]:
pd.DataFrame?

In [138]:
def get_Trappist1data():
    """
    Masses, semimajor axes, and radii are from Grimm et al. 2018
    """
    T1data = {'name': ['b', 'c', 'd', 'e', 'f', 'g', 'h'], 
              'a': [0.01155, 0.01582, 0.02228, 0.02928, 0.03853, 0.04688, 0.06193],
             'r': [1.12, 1.10, .766, .913, 1.05, 1.15, .775],
             'm': [1.017,1.156,0.297, 0.772,0.934,1.148,0.331]}
    return pd.DataFrame(T1data, index=T1data['name'])


def plot_planetTracks(simulation, truths=None, lwRange = (2., 40.)):
    import matplotlib.colors as colors
    from matplotlib.backend_bases import GraphicsContextBase, RendererBase
 
    # dirty hack to get round line caps
    class GC(GraphicsContextBase):
        def __init__(self):
            super().__init__()
            self._capstyle = 'round'

    def custom_new_gc(self):
        return GC()
    RendererBase.new_gc = types.MethodType(custom_new_gc, RendererBase)
    
    
    from matplotlib.collections import LineCollection
    fig, ax = plt.subplots()

    # find max planet mass for color code, radius range for line width
    maxM = max([max(simulation[df]['m']) for df in simulation])
    print(maxM)
    colorNorm = colors.LogNorm(0.01, maxM)
    rRange = (min([min(simulation[df]['r']) for df in simulation]),
              max([max(simulation[df]['r']) for df in simulation]))
    
    def radius2linewidth(r):
        return (r - rRange[0])/(rRange[1] - rRange[0])*(lwRange[1] - lwRange[0]) + lwRange[0] 
    
    for df in simulation:
        planet = simulation[df]
        
        # exclude planets that never grew
        if max(planet['m']) < .2:
            print('exclude {}: too small'.format(planet.name))
            continue
        
        # restrict data to times with planet status = 0
        planet = planet[planet['status'] == 0]
        
        t = planet['t'][::10]
        a = planet['a'][::10]
        m = planet['m'][::10]
        r = planet['r'][::10]
        
        # linearly transform line widths into radius range
        lw = radius2linewidth(r)
        
        # create a set of line segments
        points = np.array([t, a]).T.reshape(-1, 1, 2)
        segments = np.concatenate([points[:-1], points[1:]], axis=1)
        lc = LineCollection(segments, linewidths=lw, cmap='inferno_r', norm=colorNorm)
        lc.set_array(m)
        ax.add_collection(lc)
    
    if not truths is None:
        ax.scatter([max(t) + 1e6 for i in range(len(truths))], truths['a'], s=radius2linewidth(truths['r']), 
                   c=truths['m'], cmap='inferno_r', norm=colorNorm)
        
    # add a colorbar
    cbar = fig.colorbar(lc)
    cbar.set_label('Planet Mass [$\mathrm{M_{Earth}}$]')
    
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlabel('Time [yr]')
    ax.set_ylabel('Semimajor Axis [au]')

    return fig, ax

In [160]:
truths = get_Trappist1data()
fig, ax = plot_planetTracks(T1sim, truths)
# fig.savefig('/home/schlecker/Desktop/T1.svg', rasterized=True)

3.0695888
exclude planet001: too small
exclude planet002: too small
exclude planet003: too small
exclude planet004: too small
exclude planet005: too small
exclude planet007: too small
exclude planet008: too small
exclude planet009: too small
exclude planet010: too small
exclude planet011: too small
exclude planet012: too small
exclude planet013: too small
exclude planet014: too small
exclude planet015: too small
exclude planet016: too small
exclude planet017: too small
exclude planet018: too small
exclude planet019: too small
exclude planet020: too small
exclude planet021: too small
exclude planet022: too small
exclude planet023: too small
exclude planet025: too small
exclude planet026: too small
exclude planet027: too small
exclude planet028: too small
exclude planet029: too small
exclude planet030: too small
exclude planet031: too small
exclude planet032: too small
exclude planet033: too small
exclude planet035: too small
exclude planet036: too small
exclude planet037: too small
excl

In [185]:
# ax.scatter([7e6 for i in range(len(truths))], truths['a'])
plt.scatter([x*9e5 for x in range(8)], [x*.4 for x in range(8)], s=40, edgecolors='white', linewidth=1.5, zorder=10)
# plt.scatter([x*8e5 for x in range(8)], [x*.5 for x in range(8)], s=40, zorder=10)